In [1]:
# In this notebook, we will replace in stroke 0 by 1 and 1 by 0. After that, we will train all the model to compare the result with pre viously result

In [2]:
import pandas as pd
import exploratory_analysis
import preprocessing
import to_display_data 
import  evaluation 
import to_compute_model
import to_resample
from joblib import dump,load
import numpy as np
import to_concatenate

C:\Users\Ninette HOUKPONOU\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [4]:
model_resume=exploratory_analysis.import_data(r"C:\Users\Ninette HOUKPONOU\Repertoire_python\Memoire\new_data\model_resume.csv")

# test of previously model with replacing data

## replace 1 by 0 and vice versa in the dataset

In [6]:
data=exploratory_analysis.import_data(r"C:\Users\Ninette HOUKPONOU\Repertoire_python\Memoire\new_data\full_data.csv")

In [7]:
data.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
2,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
3,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
4,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


In [8]:
data.stroke.value_counts(normalize=True)

stroke
0    0.950211
1    0.049789
Name: proportion, dtype: float64

In [13]:
data_replace=data.copy()

In [14]:
data_replace["stroke"]=data_replace.stroke.apply(lambda x:0 if x==1 else 1)

In [15]:
data_replace.stroke.value_counts(normalize=True)

stroke
1    0.950211
0    0.049789
Name: proportion, dtype: float64

# Encodage

In [16]:
for col in data_replace.select_dtypes("object"):
    data_replace=preprocessing.to_encoded(data_replace,col)

# Split

In [17]:
xr_train,xr_test, yr_train, yr_test=preprocessing.separation_of_train_test(data_replace,"stroke",size_=0.3)

In [18]:
xr_train.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
3190,1,71.0,1,1,1,2,0,67.06,26.7,3
478,1,57.0,0,0,1,1,1,89.44,26.7,2
2083,0,16.0,0,0,0,1,0,94.03,25.7,2
911,0,29.0,0,0,0,1,1,73.67,21.0,0
2797,0,55.0,1,0,1,1,1,202.67,40.4,1


# standardization

In [19]:
standre_train=xr_train[["age","avg_glucose_level","bmi"]]
standre_test=xr_test[["age","avg_glucose_level","bmi"]]
standre_train,scaler=preprocessing.to_standardized(standre_train)
standre_test=scaler.transform(standre_test)
standre_test=pd.DataFrame(standre_test,columns=standre_train.columns)

In [20]:
xr_train=to_concatenate.to_concatenate(xr_train,standre_train)
xr_test=to_concatenate.to_concatenate(xr_test,standre_test)

In [21]:
xr_test.head()

,gender,hypertension,heart_disease,ever_married,work_type,Residence_type,smoking_status,age,avg_glucose_level,bmi
0,1,0,0,1,0,0,2,0.450684,0.115779,0.484241
1,0,0,0,0,1,0,0,0.218750,0.115132,0.510029
2,0,0,0,1,1,1,2,0.487305,0.078478,0.687679
3,0,0,0,1,1,0,2,0.768066,0.141677,0.618911
4,0,0,0,1,1,0,1,0.670410,0.004570,0.527221


In [24]:
xr_train.head()

,avg_glucose_level,bmi,age,smoking_status
0,0.055120,0.363897,0.865723,3
1,0.158434,0.363897,0.694824,2
2,0.179623,0.335244,0.194336,2
3,0.085634,0.200573,0.353027,0
4,0.681147,0.756447,0.670410,1


In [23]:
selected_columns=["avg_glucose_level","bmi","age","smoking_status"]
xr_train=xr_train[selected_columns]
xr_test=xr_test[selected_columns]

In [26]:
xr_test_resampled,yr_test_resampled,resampler=to_resample.resampling_data(xr_test,yr_test,"SMOTE")

In [12]:
loaded_rdf=load(filename="rdf_28_06.pkl")
loaded_xgb=load(filename="xgb_28_06.pkl")
loaded_bbc=load(filename="bbc_28_06.pkl")
loaded_brdf=load(filename="brdf_28_06.pkl")

In [21]:
evaluation.evaluation_of_model(loaded_rdf,xr_test_resampled,yr_test_resampled,"rdf_test_with_replacing_data",model_resume)

              precision    recall  f1-score   support

           0       0.15      0.12      0.14      1421
           1       0.25      0.29      0.27      1421

    accuracy                           0.21      2842
   macro avg       0.20      0.21      0.20      2842
weighted avg       0.20      0.21      0.20      2842



,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,metrics,RandomForest_28_06,XGBoost_28_06,BalancedBaggingClass_28_06,BalancedRandom_28_06,Random_data_smoted_28_06,XGB_data_smoted_28_06,BalancedBagsmoted_28_06,BalancedRandomsampled_28_06,rdf_test_with_replacing_data
0,0,0,0,0,0,0,precision0,0.9921722113502935,0.9661150512214342,0.976027397260274,1.0,0.8981399468556245,0.6217038539553753,0.7107231920199502,1.0,0.14899
1,1,1,1,1,1,1,recall0,0.7135819845179451,0.8627726952850105,0.8022519352568613,0.08092892329345532,0.7135819845179451,0.8627726952850105,0.8022519352568613,0.08092892329345532,0.12456
2,2,2,2,2,2,2,f1_0,0.8301268931641423,0.9115241635687733,0.880648899188876,0.14973958333333334,0.7952941176470588,0.7226643088712054,0.7537190082644629,0.14973958333333334,0.135684
3,3,3,3,3,3,3,precision1,0.13953488372093023,0.13716814159292035,0.14067278287461774,0.0536231884057971,0.762405137186223,0.7758620689655172,0.7730210016155089,0.521085441877521,0.247884
4,4,4,4,4,4,4,recall1,0.8918918918918919,0.4189189189189189,0.6216216216216216,1.0,0.9190710767065446,0.47501759324419424,0.673469387755102,1.0,0.288529
5,5,5,5,5,5,5,f1_1,0.24131627056672758,0.20666666666666664,0.22942643391521197,0.10178817056396147,0.8334396936821953,0.5892623308598866,0.7198194810078977,0.6851494696239151,0.266667
6,6,6,6,6,6,6,accuracy_,0.7224080267558528,0.8408026755852843,0.7933110367892977,0.12642140468227425,0.8163265306122449,0.6688951442646024,0.7378606615059817,0.5404644616467277,0.206545
7,7,7,7,7,7,7,macro_precision,0.5658535475356119,0.5516415964071772,0.5583500900674458,0.5268115942028986,0.8302725420209238,0.6987829614604463,0.7418720968177295,0.7605427209387605,0.198437
8,8,8,8,8,8,8,macro_recall,0.8027369382049185,0.6408458071019647,0.7119367784392414,0.5404644616467277,0.8163265306122449,0.6688951442646024,0.7378606615059817,0.5404644616467277,0.206545
9,9,9,9,9,9,9,macro_f1,0.535721581865435,0.55909541511772,0.555037666552044,0.1257638769486474,0.814366905664627,0.6559633198655459,0.7367692446361803,0.41744452647862423,0.201175


In [22]:
evaluation.evaluation_of_model(loaded_xgb,xr_test_resampled,yr_test_resampled,"xgb_test_with_replacing_data",model_resume)

              precision    recall  f1-score   support

           0       0.35      0.46      0.40      1421
           1       0.20      0.13      0.16      1421

    accuracy                           0.30      2842
   macro avg       0.27      0.30      0.28      2842
weighted avg       0.27      0.30      0.28      2842



,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,metrics,RandomForest_28_06,XGBoost_28_06,BalancedBaggingClass_28_06,BalancedRandom_28_06,Random_data_smoted_28_06,XGB_data_smoted_28_06,BalancedBagsmoted_28_06,BalancedRandomsampled_28_06,rdf_test_with_replacing_data,xgb_test_with_replacing_data
0,0,0,0,0,0,0,precision0,0.9921722113502935,0.9661150512214342,0.976027397260274,1.0,0.8981399468556245,0.6217038539553753,0.7107231920199502,1.0,0.14899,0.348469
1,1,1,1,1,1,1,recall0,0.7135819845179451,0.8627726952850105,0.8022519352568613,0.08092892329345532,0.7135819845179451,0.8627726952850105,0.8022519352568613,0.08092892329345532,0.12456,0.464462
2,2,2,2,2,2,2,f1_0,0.8301268931641423,0.9115241635687733,0.880648899188876,0.14973958333333334,0.7952941176470588,0.7226643088712054,0.7537190082644629,0.14973958333333334,0.135684,0.39819
3,3,3,3,3,3,3,precision1,0.13953488372093023,0.13716814159292035,0.14067278287461774,0.0536231884057971,0.762405137186223,0.7758620689655172,0.7730210016155089,0.521085441877521,0.247884,0.197257
4,4,4,4,4,4,4,recall1,0.8918918918918919,0.4189189189189189,0.6216216216216216,1.0,0.9190710767065446,0.47501759324419424,0.673469387755102,1.0,0.288529,0.131597
5,5,5,5,5,5,5,f1_1,0.24131627056672758,0.20666666666666664,0.22942643391521197,0.10178817056396147,0.8334396936821953,0.5892623308598866,0.7198194810078977,0.6851494696239151,0.266667,0.157873
6,6,6,6,6,6,6,accuracy_,0.7224080267558528,0.8408026755852843,0.7933110367892977,0.12642140468227425,0.8163265306122449,0.6688951442646024,0.7378606615059817,0.5404644616467277,0.206545,0.29803
7,7,7,7,7,7,7,macro_precision,0.5658535475356119,0.5516415964071772,0.5583500900674458,0.5268115942028986,0.8302725420209238,0.6987829614604463,0.7418720968177295,0.7605427209387605,0.198437,0.272863
8,8,8,8,8,8,8,macro_recall,0.8027369382049185,0.6408458071019647,0.7119367784392414,0.5404644616467277,0.8163265306122449,0.6688951442646024,0.7378606615059817,0.5404644616467277,0.206545,0.29803
9,9,9,9,9,9,9,macro_f1,0.535721581865435,0.55909541511772,0.555037666552044,0.1257638769486474,0.814366905664627,0.6559633198655459,0.7367692446361803,0.41744452647862423,0.201175,0.278031


In [23]:
evaluation.evaluation_of_model(loaded_bbc,xr_test_resampled,yr_test_resampled,"bbc_test_with_replacing_data",model_resume)

              precision    recall  f1-score   support

           0       0.26      0.29      0.28      1421
           1       0.22      0.20      0.21      1421

    accuracy                           0.24      2842
   macro avg       0.24      0.24      0.24      2842
weighted avg       0.24      0.24      0.24      2842



,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,metrics,RandomForest_28_06,XGBoost_28_06,BalancedBaggingClass_28_06,BalancedRandom_28_06,Random_data_smoted_28_06,XGB_data_smoted_28_06,BalancedBagsmoted_28_06,BalancedRandomsampled_28_06,rdf_test_with_replacing_data,xgb_test_with_replacing_data,bbc_test_with_replacing_data
0,0,0,0,0,0,0,precision0,0.9921722113502935,0.9661150512214342,0.976027397260274,1.0,0.8981399468556245,0.6217038539553753,0.7107231920199502,1.0,0.14899,0.348469,0.26442
1,1,1,1,1,1,1,recall0,0.7135819845179451,0.8627726952850105,0.8022519352568613,0.08092892329345532,0.7135819845179451,0.8627726952850105,0.8022519352568613,0.08092892329345532,0.12456,0.464462,0.287122
2,2,2,2,2,2,2,f1_0,0.8301268931641423,0.9115241635687733,0.880648899188876,0.14973958333333334,0.7952941176470588,0.7226643088712054,0.7537190082644629,0.14973958333333334,0.135684,0.39819,0.275304
3,3,3,3,3,3,3,precision1,0.13953488372093023,0.13716814159292035,0.14067278287461774,0.0536231884057971,0.762405137186223,0.7758620689655172,0.7730210016155089,0.521085441877521,0.247884,0.197257,0.220169
4,4,4,4,4,4,4,recall1,0.8918918918918919,0.4189189189189189,0.6216216216216216,1.0,0.9190710767065446,0.47501759324419424,0.673469387755102,1.0,0.288529,0.131597,0.201267
5,5,5,5,5,5,5,f1_1,0.24131627056672758,0.20666666666666664,0.22942643391521197,0.10178817056396147,0.8334396936821953,0.5892623308598866,0.7198194810078977,0.6851494696239151,0.266667,0.157873,0.210294
6,6,6,6,6,6,6,accuracy_,0.7224080267558528,0.8408026755852843,0.7933110367892977,0.12642140468227425,0.8163265306122449,0.6688951442646024,0.7378606615059817,0.5404644616467277,0.206545,0.29803,0.244194
7,7,7,7,7,7,7,macro_precision,0.5658535475356119,0.5516415964071772,0.5583500900674458,0.5268115942028986,0.8302725420209238,0.6987829614604463,0.7418720968177295,0.7605427209387605,0.198437,0.272863,0.242295
8,8,8,8,8,8,8,macro_recall,0.8027369382049185,0.6408458071019647,0.7119367784392414,0.5404644616467277,0.8163265306122449,0.6688951442646024,0.7378606615059817,0.5404644616467277,0.206545,0.29803,0.244194
9,9,9,9,9,9,9,macro_f1,0.535721581865435,0.55909541511772,0.555037666552044,0.1257638769486474,0.814366905664627,0.6559633198655459,0.7367692446361803,0.41744452647862423,0.201175,0.278031,0.242799


In [24]:
evaluation.evaluation_of_model(loaded_brdf,xr_test_resampled,yr_test_resampled,"brdf_test_with_replacing_data",model_resume)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1421
           1       0.48      0.91      0.63      1421

    accuracy                           0.46      2842
   macro avg       0.24      0.46      0.31      2842
weighted avg       0.24      0.46      0.31      2842



,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,metrics,RandomForest_28_06,XGBoost_28_06,BalancedBaggingClass_28_06,BalancedRandom_28_06,Random_data_smoted_28_06,XGB_data_smoted_28_06,BalancedBagsmoted_28_06,BalancedRandomsampled_28_06,rdf_test_with_replacing_data,xgb_test_with_replacing_data,bbc_test_with_replacing_data,brdf_test_with_replacing_data
0,0,0,0,0,0,0,precision0,0.9921722113502935,0.9661150512214342,0.976027397260274,1.0,0.8981399468556245,0.6217038539553753,0.7107231920199502,1.0,0.14899,0.348469,0.26442,0.0
1,1,1,1,1,1,1,recall0,0.7135819845179451,0.8627726952850105,0.8022519352568613,0.08092892329345532,0.7135819845179451,0.8627726952850105,0.8022519352568613,0.08092892329345532,0.12456,0.464462,0.287122,0.0
2,2,2,2,2,2,2,f1_0,0.8301268931641423,0.9115241635687733,0.880648899188876,0.14973958333333334,0.7952941176470588,0.7226643088712054,0.7537190082644629,0.14973958333333334,0.135684,0.39819,0.275304,0.0
3,3,3,3,3,3,3,precision1,0.13953488372093023,0.13716814159292035,0.14067278287461774,0.0536231884057971,0.762405137186223,0.7758620689655172,0.7730210016155089,0.521085441877521,0.247884,0.197257,0.220169,0.477574
4,4,4,4,4,4,4,recall1,0.8918918918918919,0.4189189189189189,0.6216216216216216,1.0,0.9190710767065446,0.47501759324419424,0.673469387755102,1.0,0.288529,0.131597,0.201267,0.914145
5,5,5,5,5,5,5,f1_1,0.24131627056672758,0.20666666666666664,0.22942643391521197,0.10178817056396147,0.8334396936821953,0.5892623308598866,0.7198194810078977,0.6851494696239151,0.266667,0.157873,0.210294,0.627385
6,6,6,6,6,6,6,accuracy_,0.7224080267558528,0.8408026755852843,0.7933110367892977,0.12642140468227425,0.8163265306122449,0.6688951442646024,0.7378606615059817,0.5404644616467277,0.206545,0.29803,0.244194,0.457072
7,7,7,7,7,7,7,macro_precision,0.5658535475356119,0.5516415964071772,0.5583500900674458,0.5268115942028986,0.8302725420209238,0.6987829614604463,0.7418720968177295,0.7605427209387605,0.198437,0.272863,0.242295,0.238787
8,8,8,8,8,8,8,macro_recall,0.8027369382049185,0.6408458071019647,0.7119367784392414,0.5404644616467277,0.8163265306122449,0.6688951442646024,0.7378606615059817,0.5404644616467277,0.206545,0.29803,0.244194,0.457072
9,9,9,9,9,9,9,macro_f1,0.535721581865435,0.55909541511772,0.555037666552044,0.1257638769486474,0.814366905664627,0.6559633198655459,0.7367692446361803,0.41744452647862423,0.201175,0.278031,0.242799,0.313692


# trained of previously model with replacing data

In [25]:
loaded_rdf.fit(xr_train,yr_train)

GridSearchCV(cv=5,
             estimator=RandomForestClassifier(class_weight={0: 1,
                                                            1: 19.03448275862069},
                                              random_state=1234),
             param_grid={'max_depth': [20], 'max_leaf_nodes': [2],
                         'min_samples_split': [10], 'n_estimators': [500]},
             scoring='f1')

In [27]:
# test of model rdf fit with the 
evaluation.evaluation_of_model(loaded_rdf,xr_test_resampled,yr_test_resampled,"rdf_test_fit_with_replacing_data",model_resume)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1421
           1       0.50      1.00      0.67      1421

    accuracy                           0.50      2842
   macro avg       0.25      0.50      0.33      2842
weighted avg       0.25      0.50      0.33      2842



C:\Users\Ninette HOUKPONOU\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ninette HOUKPONOU\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ninette HOUKPONOU\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Ninette HO

,Unnamed: 0.8,Unnamed: 0.7,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,metrics,...,BalancedRandomsampled_28_06,rdf_test_with_replacing_data,xgb_test_with_replacing_data,bbc_test_with_replacing_data,brdf_test_with_replacing_data,RandomForest_18_07,Random_data_smoted_18_07,RandomForest_with_avg_18_07,Random_avg_smoted_18_07,rdf_test_fit_with_replacing_data
0,0,0,0,0,0,0,0,0,0,precision0,...,1.0,0.14898989898989898,0.3484688489968321,0.2644199611147116,0.0,0.987487969201155,0.8458367683429514,0.9669485011529593,0.6030680728667306,0.0
1,1,1,1,1,1,1,1,1,1,recall0,...,0.08092892329345532,0.12456016889514426,0.4644616467276566,0.2871217452498241,0.0,0.7220267417311752,0.7220267417311752,0.8852920478536243,0.8852920478536243,0.0
2,2,2,2,2,2,2,2,2,2,f1_0,...,0.14973958333333334,0.13568417018014567,0.3981900452488688,0.27530364372469635,0.0,0.8341463414634146,0.7790432801822325,0.9243203526818518,0.7174222982606216,0.0
3,3,3,3,3,3,3,3,3,3,precision1,...,0.521085441877521,0.2478839177750907,0.19725738396624473,0.2201693610469592,0.47757352941176473,0.1337719298245614,0.7575199508901166,0.15979381443298968,0.7843915343915344,0.5
4,4,4,4,4,4,4,4,4,4,recall1,...,1.0,0.2885292047853624,0.13159746657283602,0.20126671358198453,0.9141449683321604,0.8243243243243243,0.868402533427164,0.4189189189189189,0.41731175228712175,1.0
5,5,5,5,5,5,5,5,5,5,f1_1,...,0.6851494696239151,0.26666666666666666,0.15787252005065427,0.21029411764705883,0.627384689688481,0.230188679245283,0.8091803278688524,0.23134328358208953,0.5447864033073037,0.666667
6,6,6,6,6,6,6,6,6,6,accuracy_,...,0.5404644616467277,0.20654468684025334,0.29802955665024633,0.2441942294159043,0.4570724841660802,0.7270903010033445,0.7952146375791695,0.8622073578595317,0.651301900070373,0.5
7,7,7,7,7,7,7,7,7,7,macro_precision,...,0.7605427209387605,0.19843690838249484,0.27286311648153844,0.24229466108083542,0.23878676470588237,0.5606299495128582,0.801678359616534,0.5633711577929745,0.6937298036291325,0.25
8,8,8,8,8,8,8,8,8,8,macro_recall,...,0.5404644616467277,0.20654468684025334,0.29802955665024633,0.2441942294159043,0.4570724841660802,0.7731755330277498,0.7952146375791695,0.6521054833862716,0.651301900070373,0.5
9,9,9,9,9,9,9,9,9,9,macro_f1,...,0.41744452647862423,0.20117541842340617,0.2780312826497615,0.24279888068587757,0.3136923448442405,0.5321675103543488,0.7941118040255424,0.5778318181319706,0.6311043507839627,0.333333
